# 09. Comparison of Model System Scales

## Numerical experiments - Week 04/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

## Prerequisite libraries

In [1]:
using PyPlot
using Revise

include("./conversions.jl")
using .Conversions
include("./diffusion.jl")
using .Diffusion

## 1. Revision of scale assumptions - 3 options

The diffusion model developed so far is based on the discretisation of space using a single spore diameter as a spatial increment. This leads to a straightforward abstraction of the spore as a special node on the lattice which differs in its concentration contribution from the nodes of the extracellular medium. However, it could be argued that this discretisation is too arbitrary, too rough or leading to incoherences in the interpretation of the interface between spore and medium. For this reason, two additional alternative schemes are proposed.

First, inspired by the approach taken in the derivation of the analytical scheme in Notebook 7, it can be assumed that the spore source is of negligible size compared to the lattice resolution and that a single lattice site is a discrete box that _contains_ the spore, but their boundaries are not coincident. The spore can therefore be seen as a singular source which is not significantly large to pose an obstacle for diffusion within the box, but adds new concentration to the box with a rate based on the concentration difference between the spore and the medium. In this sense, the slow-permeating sub-system of the spore is spatially removed from the sub-system of the medium lattice, but they do communicate via the entry node assumed to contain the spore. The concentration just outside the spore, in the container box, is then used as a reference for inhibition.

A second option considers the opposite scenario --- a much more fine-grain lattice, such that the detail of the spore's spherical shape can be captured. This becomes important in observing the spatial articulation of spore packings caused by aggregation and furthermore improves the accuracy of the diffusion simulation by considering scales closer to the infinitessimal limit.

## 2. Comparative experiment

The two newly proposed options are constructed and tested against the original discretisation scheme, applying different resolutions on a lattice with the same spatial dimensions and simulating inhibitor diffusion with a single spore. The concentration at the spore is recorded and compared to the analytical solution and among the numerical schemes.

In [2]:
# Global parameters
D = 600 # microns^2/s
Ps_fit = 0.0002675353069511818 # microns/s
V_spore = 125 # microns^3
A_spore = 150 # microns^2

# Analytical solution
c_in = 1.018 # M
c_out = 0.0 # M
times = [0:0.1:1;]
c_analytical = permeation_time_dependent_analytical(c_in, c_out, times, Ps_fit, A_spore, V_spore)

MethodError: MethodError: no method matching exp(::Vector{Float64})

Closest candidates are:
  exp(!Matched::BigFloat)
   @ Base mpfr.jl:674
  exp(!Matched::Missing)
   @ Base math.jl:1580
  exp(!Matched::ComplexF16)
   @ Base math.jl:1564
  ...
